### .mat < v7.3
Try using scipy.io

In [2]:
import numpy as np
import scipy.io as spio

In [3]:
cd C:/Users/haines/Dropbox/peach/dataproc/data/level1/b1

C:\Users\haines\Dropbox\peach\dataproc\data\level1\b1


In [4]:
data = spio.loadmat('b1_ctd1_deployall.mat')
data.keys()

['salin',
 'qc',
 'press',
 '__header__',
 'gsw_SA',
 '__globals__',
 'platform',
 'depth',
 'cond',
 'dens',
 'mtime',
 'wtemp',
 '__version__',
 'config']

### First level keys of dict seem okay, but ....

In [7]:
data['depth']

array([[3.56976822],
       [3.52906765],
       [3.53303844],
       ...,
       [3.58763676],
       [3.4466738 ],
       [3.21438252]])

### Nested structures are a mess

In [5]:
data['config']

array([[(array([u'4085'], dtype='<U4'), array([[360]], dtype=uint16), array([u'seconds'], dtype='<U7'), array([[array([u'mtime'], dtype='<U5'),
        array([u'days since 0000-01-01T00:00'], dtype='<U27')],
       [array([u'wtemp'], dtype='<U5'), array([u'deg C'], dtype='<U5')],
       [array([u'press'], dtype='<U5'),
        array([u'decibars'], dtype='<U8')],
       [array([u'cond'], dtype='<U4'), array([u'S m-1'], dtype='<U5')],
       [array([u'salin'], dtype='<U5'), array([u'psu'], dtype='<U3')],
       [array([u'depth'], dtype='<U5'), array([u'm'], dtype='<U1')],
       [array([u'dens'], dtype='<U4'), array([u'kg m-3'], dtype='<U6')]],
      dtype=object))]],
      dtype=[('serial_number', 'O'), ('sample_interval', 'O'), ('sample_interval_units', 'O'), ('units', 'O')])

In [6]:
def print_mat_nested(d, indent=0, nkeys=0):
    """Pretty print nested structures from .mat files   
    Inspired by: `StackOverflow <http://stackoverflow.com/questions/3229419/pretty-printing-nested-dictionaries-in-python>`_
    """
    
    # Subset dictionary to limit keys to print.  Only works on first level
    if nkeys>0:
        d = {k: d[k] for k in d.keys()[:nkeys]}  # Dictionary comprehension: limit to first nkeys keys.

    if isinstance(d, dict):
        for key, value in d.iteritems():         # iteritems loops through key, value pairs
          print '\t' * indent + 'Key: ' + str(key)
          print_mat_nested(value, indent+1)

    if isinstance(d,np.ndarray) and d.dtype.names is not None:  # Note: and short-circuits by default
        for n in d.dtype.names:    # This means it's a struct, it's bit of a kludge test.
            print '\t' * indent + 'Field: ' + str(n)
            print_mat_nested(d[n], indent+1)

print_mat_nested(data, nkeys=10)

Key: depth
Key: salin
Key: dens
Key: __header__
Key: gsw_SA
Key: __globals__
Key: platform
	Field: id
	Field: loc
	Field: mean_water_depth
	Field: mean_water_depth_units
	Field: lat
	Field: lon
	Field: mvar
	Field: start_dn
	Field: last_dn
Key: qc
	Field: rangeTest_cond_tol
	Field: rangeTest_press_tol
	Field: rangeTest_wtemp_tol
Key: cond
Key: press


### .mat >= v7.3
Try using pytables

In [8]:
import tables

In [10]:
help(tables.open_file)

Help on function open_file in module tables.file:

open_file(filename, mode='r', title='', root_uep='/', filters=None, **kwargs)
    Open a PyTables (or generic HDF5) file and return a File object.
    
    Parameters
    ----------
    filename : str
        The name of the file (supports environment variable expansion).
        It is suggested that file names have any of the .h5, .hdf or
        .hdf5 extensions, although this is not mandatory.
    mode : str
        The mode to open the file. It can be one of the
        following:
    
            * *'r'*: Read-only; no data can be modified.
            * *'w'*: Write; a new file is created (an existing file
              with the same name would be deleted).
            * *'a'*: Append; an existing file is opened for reading and
              writing, and if the file does not exist it is created.
            * *'r+'*: It is similar to 'a', but the file must already
              exist.
    
    title : str
        If the file is t